<a href="https://colab.research.google.com/github/Wenjia1215/Wenjia_ML_DDoS_experiments/blob/main/ML_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><strong>Wenjia's 4th ML experiment</strong></h1>


7/16/2022
</br>using 16 features selected in paper "Proceedings 2020, 63, 51; doi:10.3390/proceedings2020063051"


Goal: use Traffic data to predict which one is DDoS attack and which one is normal traffic.</br>
Outline:</br>

    1, Gathering Data

    2, Preparing Data

    3, Model Selection

    4, Training

    5, Evaluation

    6, Parameter Tuning

    7, Prediction

1, Gathering Data (Done by dataset lab)</br>
2, Preparing Data 

In [1]:
!gdown --id 1V9Kn7QL6knFDB1TRSIPqm0Jne5beDHZP

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1V9Kn7QL6knFDB1TRSIPqm0Jne5beDHZP
To: /content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
100% 77.1M/77.1M [00:00<00:00, 115MB/s]


In [2]:
import pandas as pd

In [3]:
file_path = '/content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'

In [4]:
ddos_data = pd.read_csv(file_path)

In [5]:
ddos_data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


Preprocess data</br>

In [6]:
#data preprocessing
# NaN
# infinity

# Deleted
  # Flow ID
  # Source IP
  # Source Port
  # Destination IP
  # Timestamp
  # Protocol


Replacing missing values

In [7]:
ddos_data.isnull().sum().sum()

4

In [8]:
ddos_data = ddos_data.fillna(ddos_data.median())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [9]:
ddos_data.isnull().sum().sum()

0

  Replace Infinity values with "0"


In [10]:
import numpy as np
ddos_data.isin([np.inf, -np.inf]).sum().sum()


64

In [11]:
ddos_data.replace([np.inf, -np.inf], 0, inplace=True)

In [12]:
ddos_data.isin([np.inf, -np.inf]).sum().sum()

0

In [13]:
ddos_data = ddos_data.replace(to_replace = "BENIGN", value = 0)
ddos_data = ddos_data.replace(to_replace = "DDoS", value = 1)

To make sure all data meaningful,</br>
Drop missing value again.

In [14]:
#count the total amount of row
ddos_data.shape[0]

225745

In [15]:
# drop missings
ddos_data = ddos_data.dropna(axis=0)

In [16]:
#count again
ddos_data.shape[0]
# the number is same, it proves that the data preprocess is efficient.

225745

Preprocess IP address<br/>
To prevent having many features you have to encode IP addresses in your training data and treat new IP as unknown
<br/> Hot encoding

In [17]:
# Pros:
# 1, The model can more easily identify IP's that belong to the same subnet.
# 2, The number of features remains small even for a large number of distinct IP addresses in your training data.
# Reference: https://www.quora.com/How-can-I-deal-with-IP-addresses-in-machine-learning-algorithms-in-traffic-analysis-and-anomaly-detection

In [18]:
# from sklearn import preprocessing
# ip_encoder = preprocessing.LabelBinarizer()

In [19]:
# ip_encoder.fit_transform(ddos_data.Flow_ID)

In [20]:
ddos_data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [21]:
y = ddos_data[' Label']
print(y)

0         0
1         0
2         0
3         0
4         0
         ..
225740    0
225741    0
225742    0
225743    0
225744    0
Name:  Label, Length: 225745, dtype: int64


Feature selection

In [22]:
# feature selection

In [31]:
# using 16 features selected in paper "Proceedings 2020, 63, 51; doi:10.3390/proceedings2020063051"
target_features = [' Destination Port','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Fwd Packet Length Max',' Fwd Packet Length Mean','Bwd Packet Length Max',
                   ' Bwd Packet Length Mean',
                   ' Fwd Header Length', ' Bwd Header Length',' Average Packet Size',' Avg Fwd Segment Size',' Avg Bwd Segment Size',' Fwd Header Length',' Subflow Fwd Bytes',' Subflow Bwd Bytes',
                   'Init_Win_bytes_forward']

In [24]:
X = ddos_data[target_features]

In [25]:
X.head()

,Destination Port,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Fwd Header Length,Bwd Header Length,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length,Subflow Fwd Bytes,Subflow Bwd Bytes,Init_Win_bytes_forward
0,54865,12,0,6,6.0,0,0.0,40,0,9.0,6.0,0.0,40,12,0,33
1,55054,6,6,6,6.0,6,6.0,20,20,9.0,6.0,6.0,20,6,6,29
2,55055,6,6,6,6.0,6,6.0,20,20,9.0,6.0,6.0,20,6,6,29
3,46236,6,6,6,6.0,6,6.0,20,20,9.0,6.0,6.0,20,6,6,31
4,54863,12,0,6,6.0,0,0.0,40,0,9.0,6.0,0.0,40,12,0,32


4, Training


Devide data to two part, one for training, one for testing(predicting)

In [26]:
# split the data into validation data & training data, for both feature and target
from sklearn.model_selection import train_test_split
train_X, validate_X, train_y, validate_y = train_test_split(X, y, random_state = 0)

In [27]:
# define model
from sklearn.tree import DecisionTreeRegressor
ddos_model = DecisionTreeRegressor()
# fit model
ddos_model.fit(train_X, train_y)

DecisionTreeRegressor()

4.1 (Experimental) prediction

In [28]:
# # get predicted Label on validation data
ddosLabel_prediction = ddos_model.predict(validate_X)

In [29]:
print(ddosLabel_prediction)
len(ddosLabel_prediction)

[0.03731343 0.         1.         ... 1.         1.         0.        ]


56437

In [30]:
# Evaluate: Determine how accurate the model's predictions are
# by using Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mean_absolute_error(validate_y, ddosLabel_prediction)

0.00039921915569928834